## Installing required libraries

In [ ]:
!pip install --quiet --upgrade langchain langchain-community gradio pypdf langchain-openai faiss-cpu

## loading credentials from .env

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('azure_credentials.env')

EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
EMBEDDING_ENDPOINT = os.getenv("EMBEDDING_ENDPOINT")
EMBEDDING_API_VERSION = os.getenv("EMBEDDING_API_VERSION")
EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("API_VERSION")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

## Importing required libraries

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableParallel

from langchain_core.prompts import ChatPromptTemplate

from langchain_community.document_loaders import PyPDFLoader

## Defining LLMs

In [ ]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

llm_2 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

## Defining prompts

In [ ]:


prompt_ = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are an expert assistant specialized in answering specific types
         of questions—such as multiple choice, true/false, and multiple correct
          selections—on a designated topic. You have access to key contextual
           information from a book. Use this context to accurately answer the
            user’s question. If the context lacks critical detail, supplement
             your response with relevant knowledge to maintain accuracy. Provide
              a concise, precise answer that matches the format of the question
               (e.g., options for MCQs, True/False for statements), ensuring the
                response is complete and directly addresses the question.""",
    ),
    ("user", """Please answer this question based on the context
     provided. Question: {question}. Context: {context}"""),
])

simple_prompt = ChatPromptTemplate.from_template("Please answer this question with very concise explaination. Question: {question}.")

## Setting up Vector DB

vectorstore = FAISS.from_documents(book_pages, embeddings)
retriever = vectorstore.as_retriever()


## Chains

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the second chain with LLM 1
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_
    | llm
    | StrOutputParser()
)

# Define the second chain with LLM 2
rag_chain_2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_
    | llm_2  # Second LLM instance
    | StrOutputParser()
)

# Define the third chain with LLM, without RAG
chain3 = (
    simple_prompt
    | llm_2  # Second LLM instance
    | StrOutputParser()
)

# Run all chains in parallel with RunnableParallel
parallel_chain = RunnableParallel(
    {"GPT_4o_MINI": rag_chain, "GPT_4o": rag_chain_2, "GP 4o Raw": chain3}
)

